In [3]:
import pandas as pd
import numpy as np 

In [4]:
data = {
    'Outlook': ['sunny','sunny', 'overcast','rainy','rainy','rainy', 'overcast','sunny','sunny','rainy','sunny', 'overcast', 'overcast','rainy'],
    'Temp': [85,80,83,70,68,65,64,72,69,75,75,72,81,71],
    'Humidity': [85,90,78,96,80,70,65,95,70,80,70,90,75,80],
    'Wind': ['weak', 'strong','weak','weak','weak','strong','strong','weak','weak','weak','strong','strong','weak','strong'],
    'Decision': ['No','No','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']    
    
}
df = pd.DataFrame(data)

In [5]:
df.to_csv("DecisionTree.csv")

 ## ID3


In [37]:
from collections import Counter

# Define the dataset
data = pd.read_csv('DecisionTree.csv')

# Define a function to calculate entropy
def entropy(data):
    labels = data['Decision']
    value_counts = Counter(labels)
    total_count = len(labels)
    entropy = 0
    for label in value_counts:
        probability = value_counts[label] / total_count
        entropy -= probability * np.log2(probability)
#     print(data, "\n",entropy)
    return entropy

# Define a function to calculate information gain
def information_gain(data, attribute):
    entropy_before = entropy(data)
    unique_values = data[attribute].unique()
    weighted_entropy_after = 0
    total_count = len(data)

    for value in unique_values:
        subset = data[data[attribute] == value]
        prob_value = len(subset) / total_count
        weighted_entropy_after += prob_value * entropy(subset)

    return entropy_before - weighted_entropy_after

# Define a function to select the best attribute to split on
def select_best_attribute(data, attributes):
    information_gains = [(attribute, information_gain(data, attribute)) for attribute in attributes]
    best_attribute, best_gain = max(information_gains, key=lambda x: x[1])
    return best_attribute

# Define the ID3 algorithm
def id3(data, attributes, parent_data=None):
    # If all instances have the same classification, return that classification
    if len(np.unique(data['Decision'])) <= 1:
        return np.unique(data['Decision'])[0]

    # If the dataset is empty, return the classification of the majority class in the parent dataset
    elif len(data) == 0:
        return np.unique(parent_data['Decision'])[np.argmax(np.unique(parent_data['Decision'], return_counts=True)[1])]

    # If there are no more attributes to split on, return the classification of the majority class
    elif len(attributes) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]

    else:
        best_attribute = select_best_attribute(data, attributes)
        tree = {best_attribute: {}}
        for value in data[best_attribute].unique():
            sub_data = data[data[best_attribute] == value]
            sub_attributes = [attr for attr in attributes if attr != best_attribute]
            subtree = id3(sub_data, sub_attributes, data)
            tree[best_attribute][value] = subtree
        return tree

# Build the decision tree
attributes = data.columns[1:-1]  # Exclude the 'Decision' column
decision_tree = id3(data, attributes)

# Print the decision tree
print("Decision Tree:")
print(decision_tree)


Decision Tree:
{'Temp': {85: 'No', 80: 'No', 83: 'yes', 70: 'yes', 68: 'yes', 65: 'no', 64: 'yes', 72: {'Outlook': {'sunny': 'no', 'overcast': 'yes'}}, 69: 'yes', 75: 'yes', 81: 'yes', 71: 'no'}}


In [38]:
print(entropy(data))
print(info_gain(data, "Humidity"))
print(attributes)

1.287054028118727
0.7506415278096601
Index(['Outlook', 'Temp', 'Humidity', 'Wind'], dtype='object')


## C4.5

In [9]:
from collections import Counter

# Define the dataset
data = pd.read_csv('DecisionTree.csv')

# Define a function to calculate entropy
def entropy(data):
    labels = data['Decision']
    value_counts = Counter(labels)
    total_count = len(labels)
    entropy = 0
    for label in value_counts:
        probability = value_counts[label] / total_count
        entropy -= probability * np.log2(probability)
    return entropy

# Define a function to calculate information gain
def information_gain(data, attribute):
    entropy_before = entropy(data)
    unique_values = data[attribute].unique()
    weighted_entropy_after = 0
    total_count = len(data)

    for value in unique_values:
        subset = data[data[attribute] == value]
        prob_value = len(subset) / total_count
        weighted_entropy_after += prob_value * entropy(subset)

    return entropy_before - weighted_entropy_after

# Define a function to select the best attribute to split on
def select_best_attribute(data, attributes):
    information_gains = [(attribute, information_gain(data, attribute)) for attribute in attributes]
    best_attribute, best_gain = max(information_gains, key=lambda x: x[1])
    return best_attribute

# Define the C4.5 algorithm
def c45(data, attributes, parent_data=None):
    # If all instances have the same classification, return that classification
    if len(np.unique(data['Decision'])) <= 1:
        return np.unique(data['Decision'])[0]

    # If the dataset is empty, return the classification of the majority class in the parent dataset
    elif len(data) == 0:
        return np.unique(parent_data['Decision'])[np.argmax(np.unique(parent_data['Decision'], return_counts=True)[1])]

    # If there are no more attributes to split on, return the classification of the majority class
    elif len(attributes) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]

    else:
        best_attribute = select_best_attribute(data, attributes)
        tree = {best_attribute: {}}
        for value in data[best_attribute].unique():
            sub_data = data[data[best_attribute] == value]
            sub_attributes = [attr for attr in attributes if attr != best_attribute]
            subtree = c45(sub_data, sub_attributes, data)
            tree[best_attribute][value] = subtree
        return tree

# Build the C4.5 decision tree
attributes = data.columns[1:-1]  # Exclude the 'Decision' column
c45_decision_tree = c45(data, attributes)

# Print the C4.5 decision tree
print("C4.5 Decision Tree:")
print(c45_decision_tree)


C4.5 Decision Tree:
{'Temp': {85: 'No', 80: 'No', 83: 'yes', 70: 'yes', 68: 'yes', 65: 'no', 64: 'yes', 72: {'Outlook': {'sunny': 'no', 'overcast': 'yes'}}, 69: 'yes', 75: 'yes', 81: 'yes', 71: 'no'}}


## CART

In [6]:
import pandas as pd
import numpy as np

# Define the dataset
data = pd.read_csv('DecisionTree.csv')

# Filter out non-numeric attributes (excluding 'Decision')
attributes = [col for col in data.columns if col != 'Decision' and np.issubdtype(data[col].dtype, np.number)]

# Define a function to calculate the Gini impurity
def gini_impurity(data):
    labels = data['Decision']
    total_count = len(labels)
    impurity = 1.0
    for label in np.unique(labels):
        probability = len(labels[labels == label]) / total_count
        impurity -= probability**2
    return impurity

# Define a function to calculate the Gini index for a split
def gini_index(data, attribute, value):
    left_subset = data[data[attribute] <= value]
    right_subset = data[data[attribute] > value]

    left_impurity = gini_impurity(left_subset)
    right_impurity = gini_impurity(right_subset)

    total_count = len(data)
    weight_left = len(left_subset) / total_count
    weight_right = len(right_subset) / total_count

    return weight_left * left_impurity + weight_right * right_impurity

# Define a function to select the best attribute and split point
def select_best_split(data, attributes):
    best_split = None
    best_gini = 1.0

    for attribute in attributes:
        unique_values = data[attribute].unique()
        unique_values.sort()

        for i in range(1, len(unique_values)):
            value = (unique_values[i - 1] + unique_values[i]) / 2
            gini = gini_index(data, attribute, value)

            if gini < best_gini:
                best_gini = gini
                best_split = (attribute, value)

    return best_split

# Define the CART algorithm
def cart(data, attributes):
    # If all instances have the same classification, return that classification
    if len(np.unique(data['Decision'])) == 1:
        return np.unique(data['Decision'])[0]

    # If there are no more attributes to split on, return the majority class
    if len(attributes) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]

    # Otherwise, find the best split and create subtrees
    best_split = select_best_split(data, attributes)
    best_attribute, best_value = best_split

    left_data = data[data[best_attribute] <= best_value]
    right_data = data[data[best_attribute] > best_value]

    left_subtree = cart(left_data, [attr for attr in attributes if attr != best_attribute])
    right_subtree = cart(right_data, [attr for attr in attributes if attr != best_attribute])

    return {'attribute': best_attribute, 'value': best_value, 'left': left_subtree, 'right': right_subtree}

# Build the CART decision tree
cart_decision_tree = cart(data, attributes)

# Print the CART decision tree
print("CART Decision Tree:")
print(cart_decision_tree)


CART Decision Tree:
{'attribute': 'Unnamed: 0', 'value': 1.5, 'left': 'No', 'right': {'attribute': 'Temp', 'value': 73.5, 'left': {'attribute': 'Humidity', 'value': 67.5, 'left': 'yes', 'right': 'yes'}, 'right': 'yes'}}


In [31]:
# Define the decision tree structure
decision_tree = {'Temp': {85: 'No', 80: 'No', 83: 'yes', 70: 'yes', 68: 'yes', 65: 'no', 64: 'yes', 72: {'Outlook': {'sunny': 'no', 'overcast': 'yes'}}, 69: 'yes', 75: 'yes', 81: 'yes', 71: 'no'}}
# global i = 1
# Define a function to convert the decision tree to a JSON format
def decision_tree_to_json(tree):
#     i = 1
    if isinstance(tree, dict):
        
        root_attribute = list(tree.keys())[0]
        root_children = tree[root_attribute]
        json_tree = {
            f"attribute": root_attribute,
            "children": []
        }
#         i+=1
        for value, subtree in root_children.items():
            subtree_json = decision_tree_to_json(subtree)
            if isinstance(subtree_json, dict):
                json_tree["children"].append(subtree_json)
            else:
                json_tree["children"].append({"value": value, "decision": subtree})
        return json_tree
    else:
        return tree

# Convert the decision tree to JSON format
json_decision_tree = decision_tree_to_json(decision_tree)
# Define the decision tree structure
decision_tree = {'Temp': {85: 'No', 80: 'No', 83: 'yes', 70: 'yes', 68: 'yes', 65: 'no', 64: 'yes', 72: {'Outlook': {'sunny': 'no', 'overcast': 'yes'}}, 69: 'yes', 75: 'yes', 81: 'yes', 71: 'no'}}

# Define a function to convert the decision tree to a JSON format
def decision_tree_to_json(tree):
#     i= 1
    if isinstance(tree, dict):
        root_attribute = list(tree.keys())[0]
        root_children = tree[root_attribute]
        json_tree = {
            f"attribute": root_attribute,
            "children": []
        }
#         i+=1
        for value, subtree in root_children.items():
            subtree_json = decision_tree_to_json(subtree)
            if isinstance(subtree_json, dict):
                json_tree["children"].append(subtree_json)
            else:
                json_tree["children"].append({"value": value, "decision": subtree})
        return json_tree
    else:
        return tree

# Convert the decision tree to JSON format
json_decision_tree = decision_tree_to_json(decision_tree)

# Print the JSON decision tree
import json
# print(json.dumps(json_decision_tree, indent=2))

# Print the JSON decision tree
import json
# print(json.dumps(json_decision_tree, indent=4))
# print(i)

In [17]:
import json

def format_json(json_data, indent_level=0):
    formatted_json = ''
    
    if isinstance(json_data, dict):
        for key, value in json_data.items():
            formatted_json += ' ' * indent_level + f'"{key}":\n'
            formatted_json += format_json(value, indent_level + 4)
    elif isinstance(json_data, list):
        for item in json_data:
            formatted_json += ' ' * indent_level + '-\n'
            formatted_json += format_json(item, indent_level + 4)
    else:
        formatted_json += ' ' * (indent_level + 4) + json.dumps(json_data) + '\n'
    
    return formatted_json

# Example JSON data

formatted_json = format_json(c45_decision_tree)
print(formatted_json)
formatted_json = format_json(decision_tree)
print(formatted_json)


"Temp":
    "85":
            "No"
    "80":
            "No"
    "83":
            "yes"
    "70":
            "yes"
    "68":
            "yes"
    "65":
            "no"
    "64":
            "yes"
    "72":
        "Outlook":
            "sunny":
                    "no"
            "overcast":
                    "yes"
    "69":
            "yes"
    "75":
            "yes"
    "81":
            "yes"
    "71":
            "no"

"Temp":
    "85":
            "No"
    "80":
            "No"
    "83":
            "yes"
    "70":
            "yes"
    "68":
            "yes"
    "65":
            "no"
    "64":
            "yes"
    "72":
        "Outlook":
            "sunny":
                    "no"
            "overcast":
                    "yes"
    "69":
            "yes"
    "75":
            "yes"
    "81":
            "yes"
    "71":
            "no"



In [58]:
def entorpy(data):
    labels = data['Decision']
    total = len(labels)
    value_count = Counter(labels)
    entorpy = 0
    for value in value_count:
        probablity = value_count[value] / total
        entropy -= probablity * np.log2(probablity)
    return entropy
print(entropy(data))


def info_gain(data, attribute):
    entopy_before = entropy(data)
    unique_counts = data[attribute].unique()
    total = len(data[attribute])
    weighter_entropy = 0
    
    for value in unique_counts:
        subset = data[data[attribute] == value]
        prob = len(subset) / total
        weighter_entropy += prob * entropy(subset)
    return entopy_before - weighter_entropy
print(info_gain(data, "Humidity"))
        
def select_best_attribute(data, attributes):
    info_gains = [(attribute, info_gain(data, attribute)) for attribute in attributes]
    best_attr, best_gain = max(info_gains, key=lambda x: x[1])
    return best_attr
print(select_best_attribute(data, ["Humidity","Outlook", "Temp"]))

def id3(data,attributes, parent_data = None):
    if len(np.unique(data['Decision'])) <= 1:
        print(data['Decision'])
        return np.unique(data['Decision'])[0]

    # If the dataset is empty, return the classification of the majority class in the parent dataset
    elif len(data) == 0:
        return np.unique(parent_data['Decision'])[np.argmax(np.unique(parent_data['Decision'], return_counts=True)[1])]

    # If there are no more attributes to split on, return the classification of the majority class
    elif len(attributes) == 0:
        
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]
    else:
        best_attribute = select_best_attribute(data,attributes)
        tree = {best_attribute: {}} 

        for value in data[best_attribute].unique():
            sub_data = data[data[best_attribute]==value]
            sub_attr = [attr for attr in attributes if attr != best_attribute]

            subtree = id3(sub_data,sub_attr, data)
            tree[best_attribute][value] = subtree
#             print("\ndata\n",sub_data,"\nsubattr\n",sub_attr)
        return tree
        

data = pd.read_csv('DecisionTree.csv')
attributes = data.columns[1:-1]  # Exclude the 'Decision' column
decision_tree = id3(data, attributes)

# Print the decision tree
print("Decision Tree:")
print(decision_tree)

    

1.287054028118727
0.7506415278096601
Temp
0    No
Name: Decision, dtype: object
1    No
Name: Decision, dtype: object
2    yes
Name: Decision, dtype: object
3    yes
Name: Decision, dtype: object
4    yes
Name: Decision, dtype: object
5    no
Name: Decision, dtype: object
6    yes
Name: Decision, dtype: object
7    no
Name: Decision, dtype: object
11    yes
Name: Decision, dtype: object
8    yes
Name: Decision, dtype: object
9     yes
10    yes
Name: Decision, dtype: object
12    yes
Name: Decision, dtype: object
13    no
Name: Decision, dtype: object
Decision Tree:
{'Temp': {85: 'No', 80: 'No', 83: 'yes', 70: 'yes', 68: 'yes', 65: 'no', 64: 'yes', 72: {'Outlook': {'sunny': 'no', 'overcast': 'yes'}}, 69: 'yes', 75: 'yes', 81: 'yes', 71: 'no'}}


In [29]:
print(data[data["Humidity"]==72])
print(data["Humidity"].unique())


Empty DataFrame
Columns: [Unnamed: 0, Outlook, Temp, Humidity, Wind, Decision]
Index: []
[85 90 78 96 80 70 65 95 75]


In [54]:
import pandas as pd
import numpy as np
from collections import Counter
import math

# Define the entropy function
def entropy(data):
    labels = data['Decision']
    total = len(labels)
    value_count = Counter(labels)
    entropy = 0  # Fixed variable name typo: entropy
    for value in value_count:
        probability = value_count[value] / total
        entropy -= probability * math.log2(probability)  # Used math.log2 instead of np.log2
    return entropy

# Define the info_gain function
def info_gain(data, attribute):
    entropy_before = entropy(data)  # Fixed variable name typo: entropy_before
    unique_counts = data[attribute].unique()
    total = len(data[attribute])
    weighted_entropy = 0  # Fixed variable name typo: weighted_entropy
    
    for value in unique_counts:
        subset = data[data[attribute] == value]
        prob = len(subset) / total
        weighted_entropy += prob * entropy(subset)
    return entropy_before - weighted_entropy

# Define the select_best_attribute function
def select_best_attribute(data, attributes):
    info_gains = [(attribute, info_gain(data, attribute)) for attribute in attributes]
    best_attr, best_gain = max(info_gains, key=lambda x: x[1])
    return best_attr

# Define the id3 function
def id3(data, attributes, parent_data=None):
    if len(np.unique(data['Decision'])) <= 1:
        return np.unique(data['Decision'])[0]

    elif len(data) == 0:
        return np.unique(parent_data['Decision'])[np.argmax(np.unique(parent_data['Decision'], return_counts=True)[1])]

    elif len(attributes) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]
                                           
    else:
        best_attribute = select_best_attribute(data, attributes)
        tree = {best_attribute: {}}

        for value in data[best_attribute].unique():
            sub_data = data[data[best_attribute] == value]
            sub_attr = [attr for attr in attributes if attr != best_attribute]

            subtree = id3(sub_data, sub_attr, data)
            tree[best_attribute][value] = subtree
            print("\ndata\n",sub_data,"\nsubattr\n",sub_attr)
        return tree

# Load your dataset
data = pd.read_csv('DecisionTree.csv')
attributes = data.columns[1:-1]  # Exclude the 'Decision' column
decision_tree = id3(data, attributes)

# Print the decision tree
print("Decision Tree:")
print(decision_tree)



data
    Unnamed: 0 Outlook  Temp  Humidity  Wind Decision
0           0   sunny    85        85  weak       No 
subattr
 ['Outlook', 'Humidity', 'Wind']

data
    Unnamed: 0 Outlook  Temp  Humidity    Wind Decision
1           1   sunny    80        90  strong       No 
subattr
 ['Outlook', 'Humidity', 'Wind']

data
    Unnamed: 0   Outlook  Temp  Humidity  Wind Decision
2           2  overcast    83        78  weak      yes 
subattr
 ['Outlook', 'Humidity', 'Wind']

data
    Unnamed: 0 Outlook  Temp  Humidity  Wind Decision
3           3   rainy    70        96  weak      yes 
subattr
 ['Outlook', 'Humidity', 'Wind']

data
    Unnamed: 0 Outlook  Temp  Humidity  Wind Decision
4           4   rainy    68        80  weak      yes 
subattr
 ['Outlook', 'Humidity', 'Wind']

data
    Unnamed: 0 Outlook  Temp  Humidity    Wind Decision
5           5   rainy    65        70  strong       no 
subattr
 ['Outlook', 'Humidity', 'Wind']

data
    Unnamed: 0   Outlook  Temp  Humidity    Wind Dec

In [60]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error


In [63]:
df = pd.read_csv("data.csv")
x = df['Experience']
y = df['Salary']

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'